In [1]:
# Import needed dependencies
import requests
import pandas as pd
import numpy as np
import scipy.stats as stats
import matplotlib.pyplot as plt
from datetime import date
from bs4 import BeautifulSoup, Comment

In [2]:
today = date.today()

# dd/mm/YY
current_year = today.strftime("%Y")

last_year = int(current_year) - 1

In [3]:
last_five_years = []
for i in range(0,5):
    last_five_years.append(int(current_year) - i)

In [4]:
# Create a list to help create a dataframe from batter statistics data
nba_stats = []

# Create a loop to create a dataframe from Basketball Reference tables
for year in last_five_years:
    
    # input URL and use BeautifulSoup to parse through the page
    url = f'https://www.basketball-reference.com/leagues/NBA_{year}_totals.html'
    soup = BeautifulSoup(requests.get(url).content, 'html.parser')

    # Grab the table element that has NBA player statistics
    table = soup.select_one('#div_totals_stats') 

    # Grab data from table and put it into the list created above
    for tr in table.select('tr:has(td)'):
        tds = [td.get_text(strip=True) for td in tr.select('td')]
        tds.append(year)
        nba_stats.append(tds)

In [5]:
# Create dataframe for batter statistics
nba_stats_df = pd.DataFrame(nba_stats)

# Create an empty list to store dataframe header information
header_list = []

# Grab the table header information to use as column headers in our dataframe
for tr in table.select('tr:has(th)'):
    ths = [th.get_text(strip=True) for th in tr.select('th')]
    header_list.append(ths)

# For loop returns a list of lists, and we only need the first list 
df_headers = header_list[0]

# Remove the first item from our headers list, it is the index header that we do not need
df_headers.remove('Rk')
df_headers.append("Year")

# Set column headers equal to our list
nba_stats_df.columns = df_headers

In [6]:
# Change types of columns to numeric for columns with number values
nba_stats_df[['Age', 'G', 'GS', 'MP', 'FG', 'FGA', 'FG%', '3P', '3PA', '3P%', '2P', '2PA', '2P%', 'eFG%', 'FT', 'FTA', 'FT%', 'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS']] = nba_stats_df[['Age', 'G', 'GS', 'MP', 'FG', 'FGA', 'FG%', '3P', '3PA', '3P%', '2P', '2PA', '2P%', 'eFG%', 'FT', 'FTA', 'FT%', 'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS']].apply(pd.to_numeric)

# Drop all players with NaN games to remove null values
nba_stats_df.dropna(subset=['G'], axis = 0 , inplace= True)

# Remove any players with fewer than 27 games played (1/3rd of the season)
filtered_nba_stats_df = nba_stats_df[nba_stats_df['G'] >= 27]

# points, rebounds, assists, steals, blocks, threes, field-goal percentage, and free-throw percentage

# Select the columns we want for our NBA player analysis
yahoo_nba_df = filtered_nba_stats_df[['Year','Player','Age','Pos','Tm','G','FG%','3P','FT%','TRB','AST','STL','BLK','PTS']]


In [7]:
# Sort by index to prepare to drop duplicates
yahoo_nba_df = yahoo_nba_df.sort_index()

# Drop duplicate entries of Player Name and Year
# This is to eliminate partial season data for players who played for 2+ teams in one season
yahoo_nba_df = yahoo_nba_df.drop_duplicates(subset=['Year', 'Player'])

In [8]:
# Sort data by name alphabetically, then by year in descending order
final_yahoo_nba_df = yahoo_nba_df.sort_values(['Year','Player'], ascending=[True, True])
final_yahoo_nba_df

,Year,Player,Age,Pos,Tm,G,FG%,3P,FT%,TRB,AST,STL,BLK,PTS
2955,2018,Aaron Brooks,33,PG,MIN,32,0.406,11,0.727,17,20,6,0,75
3092,2018,Aaron Gordon,22,PF,ORL,58,0.434,115,0.698,457,136,59,45,1022
3323,2018,Abdel Nader,24,SF,BOS,48,0.336,23,0.590,71,26,15,10,146
3155,2018,Al Horford,31,C,BOS,72,0.489,97,0.783,530,339,43,78,927
3184,2018,Al Jefferson,33,C,IND,36,0.534,0,0.833,143,30,16,23,252
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
766,2022,Yuta Watanabe,27,SF,TOR,38,0.406,27,0.600,92,21,11,16,163
136,2022,Zach Collins,24,C,SAS,28,0.490,14,0.800,153,61,13,23,218
442,2022,Zach LaVine,26,SG,CHI,67,0.476,185,0.853,308,303,41,23,1635
553,2022,Zeke Nnaji,21,PF,DEN,41,0.516,37,0.631,147,17,16,13,270


In [9]:
# Create a new dataframe for stats percentile calculations
percentile_df = pd.DataFrame(columns = ['Year', 'Player', 'Age', 'Pos', 'Tm', '3P_Percentile', 'FG%_Percentile', 'TRB_Percentile', 'FT%_Percentile', 'AST_Percentile', 'STL_Percentile', 'BLK_Percentile', 'PTS_Percentile'])

# Carry over your non-numeric columnns from final_yahoo_nba_df
percentile_df['Year'] = final_yahoo_nba_df['Year']
percentile_df['Player'] = final_yahoo_nba_df['Player']
percentile_df['Age'] = final_yahoo_nba_df['Age']
percentile_df['Pos'] = final_yahoo_nba_df['Pos']
percentile_df['Tm'] = final_yahoo_nba_df['Tm']

# Calculate the percentile rank for each player in each season, seperately, then add all the seasons in one dataframe
for year in last_five_years:
    year_df = final_yahoo_nba_df.loc[final_yahoo_nba_df['Year'] == year]
    year_df['3P_Percentile'] = year_df['3P'].rank(pct=True)
    year_df['FG%_Percentile'] = year_df['FG%'].rank(pct=True)
    year_df['TRB_Percentile'] = year_df['TRB'].rank(pct=True)
    year_df['FT%_Percentile'] = year_df['FT%'].rank(pct=True)
    year_df['AST_Percentile'] = year_df['AST'].rank(pct=True)
    year_df['STL_Percentile'] = year_df['STL'].rank(pct=True)
    year_df['BLK_Percentile'] = year_df['BLK'].rank(pct=True)
    year_df['PTS_Percentile'] = year_df['PTS'].rank(pct=True)
    year_df.sort_values('Player', ascending=True)

    # Each of the seasons are added back to the percentile dataframe
    percentile_df = percentile_df.append(year_df, ignore_index=True)



/var/folders/2d/w0_y9zt547v2y0371_wclbcm0000gn/T/ipykernel_99656/113467155.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  year_df['3P_Percentile'] = year_df['3P'].rank(pct=True)
/var/folders/2d/w0_y9zt547v2y0371_wclbcm0000gn/T/ipykernel_99656/113467155.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  year_df['FG%_Percentile'] = year_df['FG%'].rank(pct=True)
/var/folders/2d/w0_y9zt547v2y0371_wclbcm0000gn/T/ipykernel_99656/113467155.py:16: SettingWithCopyWarning: 
A value is trying to be set on a co

In [10]:
# Clean up the percentile dataframe, drop NaNs and remove unnecessary columns
percentile_df = percentile_df.dropna()
percentile_df = percentile_df.drop(['Tm','G','FG%','3P','FT%','TRB','AST','STL','BLK','PTS'], axis = 1)

# Add a rank column that adds the percentiles from each category
# Average out percentile categories for FG% and FT% with PTS, as we might still want a player with somewhat  
## lower FG% and FT% if they score a lot of points
percentile_df['Rank'] = (percentile_df['3P_Percentile'] + ((percentile_df['FG%_Percentile'] + percentile_df['PTS_Percentile']) / 2) + percentile_df['TRB_Percentile'] + ((percentile_df['FT%_Percentile'] + percentile_df['PTS_Percentile']) / 2) + percentile_df['AST_Percentile'] + percentile_df['STL_Percentile'] + percentile_df['BLK_Percentile'] + percentile_df['PTS_Percentile'])


In [11]:
# Create a list of each unique player we have in our dataframe
player_list = percentile_df.Player.unique().tolist()

# Create a new dataframe for combined, averaged percentiles over the past 5 seasons
new_df = pd.DataFrame(columns = ['Player', 'Rank', 'Trend', 'Pos', 'Years', '3P_Percentile', 'FG%_Percentile', 'TRB_Percentile', 'FT%_Percentile', 'AST_Percentile', 'STL_Percentile', 'BLK_Percentile', 'PTS_Percentile'])

# Update new dataframe with unique player list
new_df['Player'] = player_list

# Create a list for each percentile stat category for upcoming loop
player_trends = []
average_3P = []
average_FG = []
average_TRB = []
average_FT = []
average_AST = []
average_STL = []
average_BLK = []
average_PTS = []
average_Rank = []
year_count = []
pos = []

# Loop through each player, locate their percentile stats for each season, average them out
for player in player_list:
    player_df = percentile_df.loc[percentile_df['Player'] == player]
    
    # We want to find the slope of the line of best fit for each player's overall ranking each season
    x = np.array(player_df['Year'], dtype = float)
    y = np.array(player_df['Rank'], dtype = float)
    slope, intercept = np.polyfit(x, y, 1)
    player_trends.append(slope)
    
    # Find average of each player's percentiles from previous 5 seasons
    average_3P.append(sum(player_df['3P_Percentile']) / len(player_df['3P_Percentile']))
    average_FG.append(sum(player_df['FG%_Percentile']) / len(player_df['FG%_Percentile']))
    average_TRB.append(sum(player_df['TRB_Percentile']) / len(player_df['TRB_Percentile']))
    average_FT.append(sum(player_df['FT%_Percentile']) / len(player_df['FT%_Percentile']))
    average_AST.append(sum(player_df['AST_Percentile']) / len(player_df['AST_Percentile']))
    average_STL.append(sum(player_df['STL_Percentile']) / len(player_df['STL_Percentile']))
    average_BLK.append(sum(player_df['BLK_Percentile']) / len(player_df['BLK_Percentile']))
    average_PTS.append(sum(player_df['PTS_Percentile']) / len(player_df['PTS_Percentile']))
    average_Rank.append(sum(player_df['Rank']) / len(player_df['Rank']))
    year_count.append(len(x))
    
    # Keep player positions for reference purposes during the draft
    pos.append(player_df['Pos'].unique())

# Update new dataframe with the list data from each stat
new_df['Pos'] = pos
new_df['Trend'] = player_trends
new_df['3P_Percentile'] = average_3P
new_df['FG%_Percentile'] = average_FG
new_df['TRB_Percentile'] = average_TRB
new_df['FT%_Percentile'] = average_FT
new_df['AST_Percentile'] = average_AST
new_df['STL_Percentile'] = average_STL
new_df['BLK_Percentile'] = average_BLK
new_df['PTS_Percentile'] = average_PTS
new_df['Rank'] = average_Rank

# Keep track of how many seasons are being considered, so we know how reliable the data is
new_df['Years'] = year_count



/Users/michaelbinger/opt/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: RankWarning: Polyfit may be poorly conditioned
  exec(code_obj, self.user_global_ns, self.user_ns)
/Users/michaelbinger/opt/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: RankWarning: Polyfit may be poorly conditioned
  exec(code_obj, self.user_global_ns, self.user_ns)
/Users/michaelbinger/opt/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: RankWarning: Polyfit may be poorly conditioned
  exec(code_obj, self.user_global_ns, self.user_ns)
/Users/michaelbinger/opt/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: RankWarning: Polyfit may be poorly conditioned
  exec(code_obj, self.user_global_ns, self.user_ns)
/Users/michaelbinger/opt/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: RankWarning: Polyfit may be poorly conditioned
  exec(code_obj, self.user_global_ns, self.user

/Users/michaelbinger/opt/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: RankWarning: Polyfit may be poorly conditioned
  exec(code_obj, self.user_global_ns, self.user_ns)
/Users/michaelbinger/opt/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: RankWarning: Polyfit may be poorly conditioned
  exec(code_obj, self.user_global_ns, self.user_ns)
/Users/michaelbinger/opt/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: RankWarning: Polyfit may be poorly conditioned
  exec(code_obj, self.user_global_ns, self.user_ns)
/Users/michaelbinger/opt/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: RankWarning: Polyfit may be poorly conditioned
  exec(code_obj, self.user_global_ns, self.user_ns)
/Users/michaelbinger/opt/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: RankWarning: Polyfit may be poorly conditioned
  exec(code_obj, self.user_global_ns, self.user

/Users/michaelbinger/opt/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: RankWarning: Polyfit may be poorly conditioned
  exec(code_obj, self.user_global_ns, self.user_ns)
/Users/michaelbinger/opt/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: RankWarning: Polyfit may be poorly conditioned
  exec(code_obj, self.user_global_ns, self.user_ns)
/Users/michaelbinger/opt/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: RankWarning: Polyfit may be poorly conditioned
  exec(code_obj, self.user_global_ns, self.user_ns)
/Users/michaelbinger/opt/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: RankWarning: Polyfit may be poorly conditioned
  exec(code_obj, self.user_global_ns, self.user_ns)
/Users/michaelbinger/opt/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: RankWarning: Polyfit may be poorly conditioned
  exec(code_obj, self.user_global_ns, self.user

In [12]:
# Create a weighted rank column by adding trend data to the rank data and account for number of seasons played
# Basically, if you played all 5 seasons, your trend stat is added directly
# If you played fewer than all 5 seasons, your trend stat is reduced depending on how few seasons you played
new_df['Weighted Rank'] = (new_df['Rank'] + ((new_df['Trend'] * (new_df['Years'] - 1) / 4)))

# shift column 'Weighted Rank' to first position
first_column = new_df.pop('Weighted Rank')
  
# insert column using insert(position,column_name,first_column) function
new_df.insert(1, 'Weighted Rank', first_column)

new_df = new_df.sort_values('Weighted Rank', ascending = False)
new_df.head(50)

,Player,Weighted Rank,Rank,Trend,Pos,Years,3P_Percentile,FG%_Percentile,TRB_Percentile,FT%_Percentile,AST_Percentile,STL_Percentile,BLK_Percentile,PTS_Percentile
312,Nikola Jokić,7.293490,7.229375,0.064116,[C],5,0.707108,0.850647,0.987655,0.748442,0.985404,0.948232,0.872614,0.964408
191,Jayson Tatum,7.099159,6.857182,0.241977,"[SF, PF]",5,0.892891,0.564035,0.898168,0.783188,0.777362,0.903351,0.844353,0.933723
180,James Harden,7.000622,7.152452,-0.151829,"[PG, PG-SG, SG]",5,0.939523,0.460049,0.864163,0.885841,0.989090,0.933528,0.825727,0.963738
143,Giannis Antetokounmpo,6.914292,6.904490,0.009802,[PF],5,0.565308,0.888105,0.983345,0.292890,0.941954,0.887239,0.959300,0.988424
272,Luka Dončić,6.911124,6.762899,0.197633,"[PG, SG]",4,0.955300,0.556933,0.943542,0.365488,0.976297,0.854443,0.623516,0.974295
313,Nikola Vučević,6.865272,6.701964,0.163308,[C],5,0.743076,0.705396,0.974494,0.602869,0.831097,0.797233,0.898104,0.901915
228,Karl-Anthony Towns,6.826070,6.845916,-0.019847,[C],5,0.840424,0.818866,0.950154,0.747798,0.802015,0.675441,0.930512,0.932019
455,Kawhi Leonard,6.804561,6.780120,0.048883,[SF],3,0.786924,0.757865,0.843207,0.903278,0.837730,0.958610,0.626430,0.948323
365,Stephen Curry,6.791723,6.705942,0.114374,[PG],4,0.994109,0.630533,0.734059,0.980854,0.932163,0.907780,0.394058,0.969041
17,Anthony Edwards,6.758378,6.685627,0.291006,[SG],2,0.957215,0.337804,0.795596,0.511823,0.838297,0.961470,0.805826,0.951205


In [13]:
# new_df.to_csv("/Users/michaelbinger/Documents/Projects/Fantasy-Baseball-Analysis/Fantasy_Basketball/nba_trends.csv")
# final_yahoo_nba_df.to_csv("/Users/michaelbinger/Documents/Projects/Fantasy-Baseball-Analysis/Fantasy_Basketball/nba_stats.csv")


In [14]:
new_pos_list = []
pos_list = new_df['Pos'].tolist()
for item in pos_list:
    new_string = []
    for pos in item:
        string = str(pos)
        new_string = f'{new_string},{string}'
    new_pos_list.append(new_string)

final_pos_list = []
for i in new_pos_list:
    i = i.replace('[],', '')
    final_pos_list.append(i)
    
final_pos_list

new_df.drop('Pos', axis = 1, inplace = True)
new_df['Pos'] = final_pos_list

new_df

,Player,Weighted Rank,Rank,Trend,Years,3P_Percentile,FG%_Percentile,TRB_Percentile,FT%_Percentile,AST_Percentile,STL_Percentile,BLK_Percentile,PTS_Percentile,Pos
312,Nikola Jokić,7.293490,7.229375,0.064116,5,0.707108,0.850647,0.987655,0.748442,0.985404,0.948232,0.872614,0.964408,C
191,Jayson Tatum,7.099159,6.857182,0.241977,5,0.892891,0.564035,0.898168,0.783188,0.777362,0.903351,0.844353,0.933723,"SF,PF"
180,James Harden,7.000622,7.152452,-0.151829,5,0.939523,0.460049,0.864163,0.885841,0.989090,0.933528,0.825727,0.963738,"PG,PG-SG,SG"
143,Giannis Antetokounmpo,6.914292,6.904490,0.009802,5,0.565308,0.888105,0.983345,0.292890,0.941954,0.887239,0.959300,0.988424,PF
272,Luka Dončić,6.911124,6.762899,0.197633,4,0.955300,0.556933,0.943542,0.365488,0.976297,0.854443,0.623516,0.974295,"PG,SG"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
233,Keljin Blevins,0.536659,0.536659,0.000133,1,0.242788,0.009615,0.028846,0.042067,0.049279,0.106971,0.020433,0.031250,SF
453,Juwan Morgan,0.501235,0.501235,0.000124,1,0.111111,0.590123,0.014815,0.002469,0.017284,0.018519,0.038272,0.002469,PF
661,Marcus Georges-Hunt,0.480263,0.480263,0.000119,1,0.164474,0.409211,0.003947,0.088158,0.005263,0.032895,0.009211,0.007895,SG
523,Malcolm Miller,0.325858,0.325858,0.000081,1,0.149077,0.241425,0.002639,0.006596,0.007916,0.011873,0.025066,0.002639,SF


In [15]:
draft_df = new_df

In [16]:
######################################################################################################################
######################################################################################################################
######################################################################################################################
#### DRAFT DAY FUNCTIONS
   
# DROP A PLAYER 
def drafted(player):
    global draft_df
    draft_df = draft_df[draft_df.Player != player]
    return draft_df.head(25)
    
# FILTER PLAYERS BY POSITION
def position_filter(Pos):
    filtered_draft_df = draft_df[draft_df['Pos'].str.contains(Pos)]
    return filtered_draft_df.head(25)

# PULL STAT CATEGORY LEADERS
def stat_leaders(CAT):
    global final_yahoo_nba_df
    final_yahoo_nba_df = final_yahoo_nba_df.sort_values([CAT], ascending=[False])
    return final_yahoo_nba_df.head(25)

In [24]:
position_filter('C')

,Player,Weighted Rank,Rank,Trend,Years,3P_Percentile,FG%_Percentile,TRB_Percentile,FT%_Percentile,AST_Percentile,STL_Percentile,BLK_Percentile,PTS_Percentile,Pos
312,Nikola Jokić,7.293490,7.229375,0.064116,5,0.707108,0.850647,0.987655,0.748442,0.985404,0.948232,0.872614,0.964408,C
349,Rudy Gobert,5.382828,5.330407,0.052421,5,0.035596,0.987701,0.987438,0.157194,0.493026,0.631008,0.995855,0.807519,C
12,Andre Drummond,5.258315,5.559126,-0.300811,5,0.072420,0.841819,0.986262,0.062432,0.637052,0.935477,0.950731,0.762529,C
59,Clint Capela,4.907351,4.949175,-0.041825,5,0.035596,0.960700,0.980286,0.055191,0.344060,0.602619,0.972288,0.753191,C
202,Joel Embiid,6.745065,6.570410,0.174656,5,0.602249,0.739343,0.974559,0.647598,0.770662,0.660812,0.961258,0.953700,C
313,Nikola Vučević,6.865272,6.701964,0.163308,5,0.743076,0.705396,0.974494,0.602869,0.831097,0.797233,0.898104,0.901915,C
106,Domantas Sabonis,6.101364,5.768009,0.333356,5,0.299767,0.877069,0.967832,0.349445,0.837600,0.681799,0.686748,0.840503,"PF-C,PF,C"
367,Steven Adams,4.970796,5.175284,-0.204488,5,0.042720,0.937408,0.967562,0.036165,0.634460,0.830565,0.898975,0.657108,C
223,Julius Randle,6.421023,6.223705,0.197318,5,0.617421,0.630867,0.965492,0.410346,0.856644,0.674291,0.700058,0.944596,"PF,C"
205,Jonas Valančiūnas,5.817882,5.495786,0.322096,5,0.329709,0.909161,0.952941,0.558550,0.552879,0.381961,0.903407,0.820516,C


In [28]:
# draft_df = draft_df.sort_values(['Weighted Rank'], ascending=[False])
# draft_df.head(50)

,Player,Weighted Rank,Rank,Trend,Years,3P_Percentile,FG%_Percentile,TRB_Percentile,FT%_Percentile,AST_Percentile,STL_Percentile,BLK_Percentile,PTS_Percentile,Pos
312,Nikola Jokić,7.293490,7.229375,0.064116,5,0.707108,0.850647,0.987655,0.748442,0.985404,0.948232,0.872614,0.964408,C
191,Jayson Tatum,7.099159,6.857182,0.241977,5,0.892891,0.564035,0.898168,0.783188,0.777362,0.903351,0.844353,0.933723,"SF,PF"
180,James Harden,7.000622,7.152452,-0.151829,5,0.939523,0.460049,0.864163,0.885841,0.989090,0.933528,0.825727,0.963738,"PG,PG-SG,SG"
143,Giannis Antetokounmpo,6.914292,6.904490,0.009802,5,0.565308,0.888105,0.983345,0.292890,0.941954,0.887239,0.959300,0.988424,PF
272,Luka Dončić,6.911124,6.762899,0.197633,4,0.955300,0.556933,0.943542,0.365488,0.976297,0.854443,0.623516,0.974295,"PG,SG"
313,Nikola Vučević,6.865272,6.701964,0.163308,5,0.743076,0.705396,0.974494,0.602869,0.831097,0.797233,0.898104,0.901915,C
228,Karl-Anthony Towns,6.826070,6.845916,-0.019847,5,0.840424,0.818866,0.950154,0.747798,0.802015,0.675441,0.930512,0.932019,C
455,Kawhi Leonard,6.804561,6.780120,0.048883,3,0.786924,0.757865,0.843207,0.903278,0.837730,0.958610,0.626430,0.948323,SF
365,Stephen Curry,6.791723,6.705942,0.114374,4,0.994109,0.630533,0.734059,0.980854,0.932163,0.907780,0.394058,0.969041,PG
17,Anthony Edwards,6.758378,6.685627,0.291006,2,0.957215,0.337804,0.795596,0.511823,0.838297,0.961470,0.805826,0.951205,SG
